In [56]:
import pandas as pd
import numpy as np
import sklearn

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn import linear_model, feature_extraction, naive_bayes
from sklearn.feature_extraction.text import TfidfVectorizer

import xgboost
from xgboost import DMatrix

1.Загрузите	датасет	по	ссылке:	
    http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/smsspamcollection.zip
    
2.Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

3.Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".


In [7]:
data = pd.read_csv("SMSSpamCollection", sep = "\t", header=None)
data.columns = ["target", "text"]
data["target"] = data.target.map({"ham":0, "spam":1})

In [8]:
data.head()

,target,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [9]:
data[1024:1030]

,target,text
1024,0,May i call You later Pls
1025,0,Hasn't that been the pattern recently crap wee...
1026,0,I have a sore throat. It's scratches when I talk
1027,0,Yes da. Any plm at ur office
1028,0,Are you not around or just still asleep? :V
1029,0,"Lol you forgot it eh ? Yes, I'll bring it in babe"


4.Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

5.Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество – ответ в этом пункте.

In [59]:
X = data.text
y = data.target

cnt_vec = feature_extraction.text.CountVectorizer()
log_reg = linear_model.LogisticRegression()

In [60]:
pipe_estimator = Pipeline(steps=[('cnt_vec', cnt_vec), ('log_reg', log_reg)])
score = cross_val_score(estimator=pipe_estimator, X=X, y=y, scoring="f1", cv=10)
print("score = {0}".format(score.mean()))

score = 0.932640298361


6.А	теперь	обучите	классификатор	на	всей	выборке	и	спрогнозируйте	с	его	
помощью	класс	для	следующих	сообщений:

 1. "FreeMsg:	Txt:	CALL	to	No:	86888	&	claim	your	reward	of	3	hours talk	time	to	us from	your	phone	now!	Subscribe6GB"
 2. "FreeMsg:	Txt:	claim	your	reward	of	3	hours	talk	time"
 3. "Have	you	visited	the	last	lecture	on	physics?"
 4. "Have	you	visited	the	last	lecture	on	physics?	Just	buy	this	book	and	you	will	have all
  materials!	Only	99\$"
 5. "Only	99$"

Выпишите	через	пробел	прогнозы	классификатора	(0	– не	спам,	1	– спам)

In [61]:
test = pd.DataFrame(["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
            "FreeMsg: Txt: claim your reward of 3 hours talk time",
            "Have you visited the last lecture on physics?",
            "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
            "Only 99$"], columns = ["text"])

In [62]:
pipe_estimator.fit(data.text, y)

print(pipe_estimator.predict(test.text))

[1 1 0 0 0]


7.Задайте	в	CountVectorizer	параметр	ngram_range=(2,2),	затем	
ngram_range=(3,3),	затем	ngram_range=(1,3).	

Во	всех	трех	случаях	измерьте	получившееся	в	кросс-валидации	значение	f1-меры,	округлите	до	второго	знака	после	точки,	и	выпишете	результаты	через пробел	в	том	же	порядке.	

В	данном	эксперименте	мы	пробовали	добавлять	в	признаки	n-граммы	для	разных	диапазонов	n	- только биграммы,	только	триграммы,	и,	наконец,	все	вместе	- униграммы,	биграммы	и	триграммы.	

Обратите	внимание,	что	статистики	по	биграммам	и	триграммам	намного	меньше,	поэтому	классификатор	
только	на	них	работает	хуже.

В	то	же	время	это	не	ухудшает	результат	сколько-нибудь	существенно,	если	добавлять	их	вместе	с	униграммами,	т.к.	за	счет	регуляризации	линейный	классификатор	не	склонен	сильно	переобучаться	на	этих	признаках.

In [63]:
ngram_ar = [(2,2), (3,3), (1,3)]
for ngram in ngram_ar:
    cnt_vec = feature_extraction.text.CountVectorizer(ngram_range=ngram)
    log_reg = linear_model.LogisticRegression()
    pipe_estimator = Pipeline(steps=[('cnt_vec', cnt_vec), ('log_reg', log_reg)])
    
    score = cross_val_score(estimator=pipe_estimator, X=X, y=y, scoring="f1", cv=10)
    print("ngram = {0}, score = {1:.2f}".format(ngram, score.mean()))

ngram = (2, 2), score = 0.82
ngram = (3, 3), score = 0.73
ngram = (1, 3), score = 0.93


8.Повторите	аналогичный	п.7	эксперимент,	используя	вместо	
логистической	регрессии	MultinomialNB().

Обратите	внимание,	насколько	сильнее	(по	сравнению	с	линейным	классификатором)	наивный	Байес страдает	от	нехватки	статистики	по	биграммам	и	триграммам.

In [64]:
ngram_ar = [(2,2), (3,3), (1,3)]
for ngram in ngram_ar:
    cnt_vec = feature_extraction.text.CountVectorizer(ngram_range=ngram)
    naive_bayes = sklearn.naive_bayes.MultinomialNB()
    pipe_estimator = Pipeline(steps=[('cnt_vec', cnt_vec), ('naive_bayes', naive_bayes)])
    
    score = cross_val_score(estimator=pipe_estimator, X=X, y=y, scoring="f1", cv=10)
    print("ngram = {0}, score = {1:.2f}".format(ngram, score.mean()))

ngram = (2, 2), score = 0.93
ngram = (3, 3), score = 0.87
ngram = (1, 3), score = 0.95


9.Попробуйте использовать в логистической регрессии в качестве признаков Tf-idf из TfidfVectorizer на униграммах.

Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? 

Обратите внимание, что результат перехода к tf-idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [65]:
tf_idf = feature_extraction.text.TfidfVectorizer(ngram_range=(1,1))
log_reg = linear_model.LogisticRegression()
pipe_estimator = Pipeline(steps=[('tf_idf', tf_idf), ('log_reg', log_reg)])

score = cross_val_score(estimator=pipe_estimator, X=X, y=y, scoring="f1", cv=10)
print("Tf-idf with unigramm = (1,1), score = {0:.2f}".format(score.mean()))

Tf-idf with unigramm = (1,1), score = 0.88


In [66]:
cnt_vec = feature_extraction.text.CountVectorizer(ngram_range=(1,1))
log_reg = linear_model.LogisticRegression()
pipe_estimator = Pipeline(steps=[('cnt_vec', cnt_vec), ('log_reg', log_reg)])
    
score = cross_val_score(estimator=pipe_estimator, X=X, y=y, scoring="f1", cv=10)
print("CountVectorizer with unigramm = (1,1), score = {0:.2f}".format(score.mean()))

CountVectorizer with unigramm = (1,1), score = 0.93


Качество кросс-валидации на униграммах понизилось при использовании Tf-idf по сравнению с CountVectorizer

10.*Попробуйте получить как	можно более	высокое	качество на кросс-валидации.
Напишите, что пробовали	и какое	качество получилось.

Я попробовал СountVectorizer c xgboost, но качество 0.945 не превзошел,варьируя ngram_range

11.Какие	наблюдения	и	выводы	можно	сделать	из	этого	задания?

Что использовать Tf-idf или CountVectorizer зависит от данного нам датасета. Точность больше при использовании униграмм, биграмм и триграмм вместе взятых, что очевидно, но иногда хорошего результата можно добиться использованием допустим только биграмм.